# Tekstikolletsioonid ja nende kasutamine

Tekstikollekstsioonide (*korpuste*)  töötlemiseks on olemas kaks põhimõttelist võimalust:

* tekstid on salvestatud failidena (*pickle*),
* tekstid on salvestatud andmebaasi objektidena.

Andmebaasi kasutamise eeliseks on parem otsitavus ning võimalus kasutada olemasolevaid andmetöötlusalgoritme.

* EstNLTK 1.4 teek kasutab [ElasticSearch](https://www.elastic.co/products/elasticsearch) otsingumootorit teksikollektsioonide salvestamiseks. 
* EstNLTK 1.6 teek kasutab [PostgreSQL](https://www.postgresql.org) andmebaasi teksikollektsioonide salvestamiseks. 

PostgreSQL eeliseks:

* stabiilne API,
* ennustatav resursi kulu,
* parem skaleeruvus praktiliste andmemahtude juures,
* lihtsam integreeritavus olemasolevasse it-taristusse.

**Tüüpilised sammud tekstikollektsioonide töötamisel** 
* Tekstikollektsiooni loomine
* Tekstikollektsiooni annoteerimine
* Tekstikollektsioonidest otsimine


## 0. Ettevalmistavad sammud 

Selleks et Jupyteriga oleks lihtsam töötada tuleks soovitusi muuta agresiivsemaks (*autocomplete*).

In [1]:
%config IPCompleter.greedy=True

See võimaldab meil *Tab*-iga küsida objekti meetodeid ning *Shift+Tab*-iga küsida funktsioonide dokumentatsiooni.

## 1. Tekstikollektsioonide loomine failidest

Selleks et oleks selge, mis mooduleid konkreetsetes sammudes kasutatakse, impordime moodulid vajadusepõhiselt iga andmetöötlussammu ees.

### Sisendtektide uurimine

Järgnevas on meie eesmärk uurida raadiosaadete automaat-transkriptsiooni abil saadud materjale. Vastavad failid asuvad kataloogis `data/kpt` ning on UTF-8 kodeeringus. Nende lugemiseks on vaja failid õiges kodeeringus avada.

In [2]:
file = open("data/kpt/2019-09-07.txt", mode="r", encoding="utf-8")
raw_text = file.read()
print(raw_text[:300])

K01: Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?
-:
K02: Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Pal


Teksti vaadates on selge, et tekst koosneb blokkidest, mis algavad kõneleja nimega ning millele millele järgneb repliik. 
Selline struktuur on väha sagedane ka ametlike dokumentide või tootekirjelduste korral, 
kus teksti eri osade eraldamiseks kasutatakse pealkirju või muid korduvaid struktuurielemente. 
Tavaliselt on kõige lihtsam selliseid struktuurielemente tuvastada regulaaravaldiste abil.  

## Teksti segmenteerimine

Esimese asjana tuleb tekstist üles leida kõnetuvastaja poolt pandud rääkija tähis või nimi. Selleks kasutame kahte märgendajat:
* ```ŖegexTagger``` abil märgime peale rääkija,
* ```TextSegmentsTagger``` abil märgime peale kõnelejale vastava repliigi. 

### Kõnelejale vastav regulaaravaldis

Kuna kõnetuvastuse väljund on selgete struktuuriga, siis on vastava regulaaravaldise leidmine üsna lihtne.
Tavaliselt on kerge leida ka muude tekstide erinevaid alamosasid eraldavate struktuurielemetidele 
(lõikude nummerdus, kuupäevaline kirje päis, jms) vastavaid mustreid. 

Igal juhul on vastava regulaaravaldise tuletamiseks ja testimiseks vaja võtta illustreerivad näited. Nendest saab hiljem luua ka ühiktesti, mille abil tulevikus tagada funktsionaalsuse säilimine edasisel mustrite täiendamisel.

In [3]:
import re

In [4]:
test_text =  'K01: Kuku...\n-:\nArtur Talvik: Tere, siin on...'

Kuna regulaaravaldistest arusaamine on keerukas, siis teeme seda sammhaaval. 
Selleks, et mitte takerduda paosümbolite (*escape symbols*) rägastikku, on mõistlik kasutada formaati `r'sõne'`.
Nii ei ole vaja regulaaravalidiste paosümboleid mitmekordselt pagendada.

In [5]:
print(re.findall(r'^.',test_text))
print(re.findall(r'((^|\n).)',test_text))
print(re.findall(r'((^|\n).*:)',test_text))

['K']
[('K', ''), ('\n-', '\n'), ('\nA', '\n')]
[('K01:', ''), ('\n-:', '\n'), ('\nArtur Talvik:', '\n')]


Et edaspidi oleks lihtsam, tuleks regulaaravaldise osadele anda nimed:
* ```name``` - kõneleja nimi,
* ```span``` - kõneleja fikseeriv tekstifragment.

In [6]:
pattern = r'((^|\n)(?P<span>(?P<name>.*): ?))'
print([match.group('name') for match in re.finditer(pattern, test_text)])
print([match.group('span') for match in re.finditer(pattern, test_text)])
print([match.group() for match in re.finditer(pattern, test_text)])

['K01', '-', 'Artur Talvik']
['K01: ', '-:', 'Artur Talvik: ']
['K01: ', '\n-:', '\nArtur Talvik: ']


### RegexTagger mustrisõnastik

Märgendaja ```RegexTagger``` defineerimiseks on peale mustrite vaja veel määrata:
* ```_regex_pattern_``` - regulaaravaldise muster,
* ```_group_``` - tekstifragmenti (*spani*) defineeriv regulaaravaldise grupp,
* ```_priority_``` - mustri prioriteet konfliktide korral,
* ```_validator_``` - validaator valepositiivsete vastete eemldamiseks.

Esimesed kaks parameetrit on alati vajalikud. Meie näites on meil märgendamiseks vajalik vaid üks muster. 

In [7]:
header_voc_1 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
}]

Lisaks on mõistlik defineerida ka tekstifragmentidele (*spanidele*) vastavad annotatsiooniatribuudid (kõneleja isik). 
Selleks tuleb anda ette funktsioon, mis võttab sisse kogu regulaaravalidsele vastava ```Match``` objekti ja tuletab sellest atribuudi väärtuse.  

In [8]:
header_voc_2 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    'person': lambda m: m.group('name') 
}]

In [9]:
header_voc_3 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_validator_': lambda m: m.group('span') != '-:', 
    'person': lambda m: m.group('name') 
}]

### Märgendaja enda loomine

Märgendaja `RegexTagger` loomisel on tarvis alati määrata kolm peamist parameetrit:

* `vocabulary` - mustrisõnastik,
* `output_layer` - väljundkihi nimi,
* `output_attributes` - väljundkihti tellitavate atribuutide nimed.

Lisaks saab mängida erinevate täpishäälestust võimaldavate parameetritega:
* `ambiguous` - kas leitud märgendused võib olla mitu annotatsiooni,
* `conflict_resolving_strategy` - mida teha kui regulaaravalistele vastavad fragmendid on ülekattes 
* `overlapped, ignore_case` - lisaargumendid argumendid `re.finditer` funktsiooni häälestamiseks

Täpishäälestuse parameetreid on enamasti vaja vaid siis, kui mustrisõnastik käitub ootamatult ning on tarvis aru saada, mis läks valesti.  

In [12]:
from estnltk import Text
from estnltk.taggers import RegexTagger

Tekstifragmente ei pea attribuutidega annoteerima. 

In [13]:
tagger = RegexTagger(vocabulary = header_voc_1, output_layer = 'headers') 
text = tagger.tag(Text(raw_text))
display(text.headers[:5])

Layer(name='headers', attributes=(), spans=SL[Span('K01: ', [{}]),
Span('-:', [{}]),
Span('K02: ', [{}]),
Span('-:', [{}]),
Span('K06: ', [{}])])

Märgendaja lisab tekstifragmentidele atribuudi ainult siis, kui:

* vastav atribuut on tellitud väljundkihti,
* mustrisõnastikus on reegel atribuudi arvutamiseks.


In [15]:
# Atribuut person pole tellitud
tagger = RegexTagger(vocabulary = header_voc_2, output_layer = 'headers') 
text = tagger.tag(Text(raw_text))
display(text.headers[:3])

# Atribuut person on tellitud 
tagger = RegexTagger(
    vocabulary = header_voc_2, 
    output_layer = 'headers',
    output_attributes = ['person']
) 

text = tagger.tag(Text(raw_text))
display(text.headers[:3])

Layer(name='headers', attributes=(), spans=SL[Span('K01: ', [{}]),
Span('-:', [{}]),
Span('K02: ', [{}])])

Layer(name='headers', attributes=('person',), spans=SL[Span('K01: ', [{'person': 'K01'}]),
Span('-:', [{'person': '-'}]),
Span('K02: ', [{'person': 'K02'}])])

### Teksti segmenteerimine erinevate inimeste kõneks

Järgmiseks loomulikuks sammuks on repliikide eraldamine tekstist. Selle käigus on mõistlik kõneleja isik panna eraldi atribuudiks.
Jällegi on tegemist standardse teksti struktureerimise ülesandega, mille käigus jagatakse põhitekst päiste (*header*) järgi osadeks. 

Selle jaoks on EstNLTK teegis olemas `TextSegmentsTagger` märgendaja, mille loomisel tuleb määrata parameetrid:

* `input_layer` - päiselementide kiht,
* `output_layer` - väljundkiht,
* `output_attributes` - väljundkihti tellitavate atribuutide nimekiri.

In [16]:
from estnltk.taggers import TextSegmentsTagger

In [17]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'])

tagger.tag(text)
text.lines[:5]

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': None}]),
Span('\n', [{'person': None}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': None}]),
Span('\n', [{'person': None}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': None}])])

Saadud tulemus on vigane.

* Repliikide hulka on sattunud ka tühje tekste.
* Kuigi `person` on tellinud väljundkihti, siis ei jõua selle väärtus atribuutide hulka.

Nende probleemide lahendamiseks tuleb `TextSegmentsTagger` loomisel määrata parameetrid:

* `validator` - funktsioon päiste täiendavaks valideerimiseks,
* `decorator` - funktsioon tellitud attribuutide väärtuste arvutamiseks.

Viimaks saab parameetriga 

* `include_header` - määrata kas päiselement kuulub tekstifragmendi koosseisu või mitte. 

Tüüpiliselt on mõistlik päisest mõelda kui tekstifragmendile vastavast metainfost ja seega see ei peaks see olema teksti osa. 

In [19]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person']},
    validator = lambda header_span: header_span['person'] != '-' 
)

if 'lines' in text.layers:
    del text.lines

tagger.tag(text)
display(text.lines[:5])
display(text.lines[0])

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n-:\n', [{'person': 'K01'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n-:\n', [{'person': 'K02'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}]),
Span('Tere, siin on Keskpäevatund ja Kuku raadio, Tallinna stuudios Urmas Jaagant, Ainar Ruussaar ja Priit Hõbemägi. Mart Sander laulis lõbusa laulukese. Palun lenda sirgelt ja see laul on meie poolt tervituseks Gunnar kobinile. Kes kelle kohta hiljuti saime teada, et ta loobus Nordika juhi ametikohast, nimelt ta pidi Kobin alustama teisel septembril tordika juhina, aga ootamatult selgus, et ta on juba suvel oma ametikohast loobunud ja tõi ühe põhjusena välja selle, et ühiskonnas tekkinud suur diskussioon tema töötasu üle, mis oli siis viisteist tuhat bruto ühes kuus ei ole vastavuses tema tegeliku töö panusena. Et seda tema lauset oskan mina kahte moodi analüüsida, üks on see, et liiga, et talle makstakse liiga palju raha ja tema panuseks väikeseks, aga teisest küljest võiks mõelda ka niimoodi, et ta ütleb, et selle panuse eest, mis tema suudaks ettevõtte arengusse panna, on seda raha liiga vähe. Kuidas see asi tegelikult on, eks teab ta ise.\n', [{'person': 'Artur Talvik'}]),
Span('Ma arvan, et asi on lihtne nagu tavapäraselt, et Gunnar Kobin ilmselt läks süsteemi mille esiesmapilgul seal tundmaõppimisel said aru, et tema natuur, tema eesmärgid ja see süsteem, kuhu ta sattus ei klapi päris hästi kokku.\n', [{'person': 'Ainar Ruussaar'}])])

text,person
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n-:\n,K01


Saadud tulemus on ikkagi vigane:

* Ignoreeritud päiselemendid lähevad alles jäänud teksti fragmentide koosseisu.

Tühjade tekstifragmentide eemaldamiseks saab kasutada funktsiooni `apply_filter`, aga hetkel jätame selle probleemi lahendamata.

In [24]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person']}
)
if 'lines' in text.layers:
    del text.lines
tagger.tag(text)
display(text.lines[:5])
display(text.lines[0])

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': 'K01'}]),
Span('\n', [{'person': '-'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': 'K02'}]),
Span('\n', [{'person': '-'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}])])

text,person
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n,K01


### Segmentatsiooni esmane valideerimine

Esmase korrektsuse hindamiseks kasutame kihi grupeerimisfunktsiooni, mis käitub analoogselt mooduli `Pandas` meetodiga `groupby`.

In [25]:
text.lines.groupby(['person']).count

{('K01',): 1,
 ('-',): 3,
 ('K02',): 1,
 ('K06',): 3,
 ('Artur Talvik',): 21,
 ('Ainar Ruussaar',): 16,
 ('K07',): 13,
 ('Ignar Fjuk',): 2}

Järgnev näide võimaldab uurida millised repliigid vastavad kõnelejale `K06`. 

In [26]:
for span in text.lines.groupby(['person']).groups[('K06',)]:
    print(span.text)


Ta oli liiga uudis ainult koduvabariigist.

Ta on tähtajaline neli päevauudiseid koduvabariigist.

Ta oli neli päevauudiseid koduvabariigist.



Viimase kõige keerukama näidena leiame repliikide kogupikkuse tähemärkides. 

In [27]:
display(text.lines.groupby(['person'])
      .aggregate(func = lambda spans: sum(s.end - s.start for s in spans)))

{('K01',): 104,
 ('-',): 2,
 ('K02',): 243,
 ('K06',): 140,
 ('Artur Talvik',): 19145,
 ('Ainar Ruussaar',): 9171,
 ('K07',): 12607,
 ('Ignar Fjuk',): 30}

Kuna Keskpäevatunnis on reeglina kolm osalejat, siis võib arvata, et märgend `K07` vastab Ignar Fjukile.

## 2. Tekstikollektsiooni loomine

Järgnevas näites lisame kõikidele repliikidele vastavad tekstiobjektid Postgre andmebaasis olevasse tekstikollektsiooni. 

### Ühenduse loomine Postgre SQL andmebaasiga 

EstNLTK teegis defineeritakse ühenduse parameetrid läbi `PostgreStorage` klassi.  

In [28]:
from estnltk.storage.postgres import PostgresStorage

In [29]:
storage = PostgresStorage(host='127.0.0.1',
                          port=5432,
                          dbname='ekt',
                          user='swen',
                          password='kala',
                          schema='media_analysis',
                          role=None,
                          temporary=False)

INFO:storage.py:42: connecting to host: '127.0.0.1', port: 5432, dbname: 'ekt', user: 'swen'
INFO:storage.py:58: schema: 'media_analysis', temporary: False, role: 'swen'


Kui andmebaasis parameetritele vastavat skeemat pole siis tuleb see luua. 
Analoogselt saab ka kogu skeema kustutada.

In [30]:
from estnltk.storage.postgres import create_schema, delete_schema

In [36]:
# create_schema(storage)
# delete_schema(storage)

Lisaks saab vaadata ka skeemasse kuuluvadi kollektsioone ning ühenduse parameetreid.

In [37]:
storage

### Tekstikollektsiooni formaadi fikseerimine

Tekstikollektsiooni saab tekstidele lisaks salvestada ka metaandmeid. 
Metaandmeid hoitakse eraldiseisvates veergudes ning edaspidi on nende järgi lihtne ja kiire tekste grupeerida.
Meie näite korral on teksti kirjeldavateks olulisteks metaatribuutideks:

* kõneleja isik, 
* saate toimumise aeg.

Neist esimene on sõne ja teine kuupäeva formaadis. Need andmeformaadid on EstNLTK poolt toetatud. 

In [38]:
# Metaatribuutide kirjeldus
storage['kpt'].meta = {'date':'datetime', 'person':'str'}
# Kollektsiooni formaadi fikseerimine
storage['kpt'].create('Keskpäevatunni saadete automaatne transcriptsioon')

INFO:collection.py:107: new empty collection 'kpt' created


,data type
date,timestamp without time zone
person,text


Implementatsioonivea tõttu ei jäta tekstikollektsioon meelde tellitud metaatribuute ning need tuleb eraldi tellida. 

In [39]:
print(storage['kpt'].column_names)
storage['kpt'].column_names += list(storage['kpt'].meta.keys())
print(storage['kpt'].column_names)

['id', 'data']
['id', 'data', 'date', 'person']


### Tekstide lisamine kollektsiooni

Tekstide lisamiseks tuleb itereerida üle kõikide originaal failide:

* luua esmane tekstobjekt,
* märgendada igale tekstile repliigid, 
* luua iga repliigi jaoks eraldi tekstobjekt ning paigutada see kollektsiooni.

Kuna eelnevalt on kogu vajalik töö ära tehtud, siis nüüd on vaja vaid kood kokku koguda.

In [40]:
import os
from datetime import datetime

In [41]:
header_voc_2 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    'person': lambda m: m.group('name') 
}]

header_tagger = RegexTagger(
    vocabulary = header_voc_2, 
    output_layer = 'headers', 
    output_attributes = ['person'])

In [42]:
segmenter = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda span: {'person': span['person']},
)

In [43]:
collection = storage['kpt']
directory = os.fsencode('data/kpt')

with collection.insert() as collection_insert:

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename == 'readme.md':
            continue
        
        print(filename)
        date = datetime.strptime(re.search(r'(?P<date>.*)\.txt', filename).group('date'),'%Y-%m-%d')
        file = codecs.open("data/kpt/{}".format(filename), "r", "utf-8")
        text = segmenter(header_tagger(Text(file.read())))
    
        for span in text.lines:
            
            line = span.text.strip()
            if line == '':
                continue
                
            line_text = Text(line)
            line_text.tag_layer(['compound_tokens', 'words', 'paragraphs'])
            meta_data = {'date': date, 'person': span['person']}
            collection_insert(text=line_text, meta_data=meta_data)

2019-09-07.txt
INFO:collection.py:325: inserted 0 texts into the collection 'kpt'


NameError: name 'codecs' is not defined

## 3. Tekstikollektsiooni annoteerimine

### Tekstikollektsiooni märgenduskihtide tüübid 

Märgenduskihid tekstikollektsioonis jagunevad kaheks:

* esialgsed kihid (luuakse koos tekstiobjektiga).
* eraldi seisvad kihid (neid saab pärast juurde teha).

Kuna esialgsete kihtide muutmine eeldab tekstiobjekti muutmist, siis on mõistliklisada tekstile vaid need kihid, mida edasiste analüüsidega ei muudeta. Kõik ülejäänid kihid tuleks lisadada hiljem eraldi. Meie näites on esialgseteks kihtideks vaid segmentatsioonikihid, mida on vaja morfoloogiliseks analüüsiks. Kõik ülejäänud kihid lisame tekstidele hiljem juurde.

Esialgseid kihte on võimalk vaadata vaadates kollektsiooni kuuluvat teksti.

In [ ]:
collection[0]

### Uue kihi lisamine tekstikollektsiooni

Tekstikollektsiooni uue kihi lisamikseks on kõige lihtsam viis, vajaliku märgendaja loomine ning selle rakendamine kogu kollektsioonile. 
Meie näiteks on selleks vabamorfile vastav märgendaja.

In [ ]:
from estnltk.taggers import VabamorfTagger 

In [ ]:
vabamorf = VabamorfTagger(disambiguate = True, output_layer = 'morph_analysis')
collection.create_layer(tagger=vabamorf)

Tulemust saab vaadata kollektsiooni vaadates. Sealjuures ei ole ilma täiendavate sammudeta kihti võimalik näha.

In [ ]:
display(collection)
display(collection[0])

### Naiivne nimeolemi tuvastus

Järgnevas näitame kuidas saab repliikide peale märkida mõned olulised nimeolemid, mille sagedust saaks näiteks hiljem ajas jälgida. Meie eesmärgiks on siin vaid süsteemi tutvustamine mitte parim täpsus. 

In [ ]:
text = Text(raw_text).analyse('morphology')
text.morph_analysis[:5]

Vaatame sagedasi lemmasid tuvastamaks oluliste fraaside põhju.

In [ ]:
lemma_freq = text.morph_analysis.groupby(['lemma']).count
sorted(lemma_freq.items(), key=lambda x: x[1], reverse=True)

In [ ]:
from estnltk.taggers import PhraseTagger

In [ ]:
entity_voc=[
{
    '_phrase_': ('kuku', 'raadio'),
    'entity': 'Kuku Raadio'
},
{
    '_phrase_': ('keskerakond',),
    'entity': 'Keskerakond' 
},
{
    '_phrase_': ('Keskerakond',),
    'entity': 'Keskerakond' 
},

{
    '_phrase_': ('Eesti',),
    'entity': 'Eesti'
},
{
    '_phrase_': ('Ukraina',),
    'entity': 'Ukraina'
}   
]

In [ ]:
entity_tagger=PhraseTagger(
    output_layer='entities',
    output_attributes=['entity'],
    input_layer='morph_analysis', 
    input_attribute='lemma',
    vocabulary=entity_voc, 
    key='_phrase_')

In [ ]:
if 'entities' in text.layers:
    del text.entities
entity_tagger(text)
text.entities.groupby(['entity']).count

Lets run the corresponding tagger on the entire text collection.

In [ ]:
collection.create_layer(tagger=entity_tagger)
collection

### Tulemuste kiire vaatamine

Selleks et tulemust näha, tuleb eraldi loodud kiht lülitada tekstobjekti rekonstrueerimisel kasutatavate kihtide loetellu. 

In [ ]:
collection.selected_layers.append('entities')
collection.selected_layers

In [ ]:
display(collection[0].text)
display(collection[0].entities)

## 4. Päringud tekstikollektsiooni

Päringuid on vaja selleks, et valida välja teatud kriteeriumidele vastavad tekstid. 
Selleks on vaja luua päringuobjekt ning rakendada seda kogu kollektsioonile või alamkollektsioonile.
Erinevaid päringuobjekte on mitmeid ning neid saab kombineerida. 

### Kihipäringud
Vaatame esmalt kihipäringuid, et tuvastada, millised nimeolemid esinevad Kuku Raadioga samas repliigis.  

In [ ]:
from estnltk.storage.postgres import JsonbLayerQuery

In [ ]:
layer_query ={'entities': JsonbLayerQuery('entities', entity='Kuku Raadio')}

In [ ]:
for key, text, meta in collection.select(
    layer_query=layer_query,
    layers=['entities'],
    return_index=True,
    collection_meta=['person', 'date']):
    display(text.entities)

### Päringud originaalsele tekstiobjektile

Esialgsetele kihtidele saab teha päringuid läbi tekstipäringute, aga siis peavad vastavad kihid sisaldama atribuute. Meie ühelgil kihil pole atribuute, kuna `text` pole päris atribuut. Seega ei saa me ühtegi huvitavat päringut teha.

In [ ]:
from estnltk.storage.postgres import JsonbTextQuery

In [ ]:
collection.select(query=JsonbTextQuery('tokens', text='raadios')).head()

### Päringud alustekstile

Leidub eraldi päringuobjekt alusteksti alamsõnede otsimiseks. Seda on kasulik kasutada otsingufaasis.

In [ ]:
from estnltk.storage.postgres import SubstringQuery

In [ ]:
collection.select(SubstringQuery('kuku')).head(2)


## 5. Andmebaasiühenduse sulgemine

Ühenduse sulgemine.

In [ ]:
storage.close()

Käsud Postgre objektide kustutamiseks.

In [ ]:
# storage.delete('kpt')
# storage['kpt'].delete()